# Loss Comparison between Numpy and Tensorflow

This document aims to compare the losses that are used in SCORE and ShapeDeconv and ultimately harmonize them.

## Load present losses

<b>Note:</b> Only the differentiable part of the loss is considered is the case of SCORE.

Setting required constants and variables:

In [1]:
# import libraries
import numpy as np
import cadmos_lib as cl
import tensorflow as tf
import sys
from numpy.linalg import norm

# declare constants that are common for single and multi window constraint
n_row = n_col = 96
gamma = 1
U = cl.makeUi(n_row,n_col)

# generate mock data
y_pred,win = np.random.rand(2,3,n_row,n_col)
y_true = np.zeros((3,n_row,n_col))
residual = y_true-y_pred

#declare constants that are specific to single window constraint
muw = np.array([y_pred.size/norm(win*Ui)**2/U.shape[0] for Ui in U])

#declare constants that are specific to single window constraint
n_shearlet = 3
shearlets,shearlets_adj = cl.get_shearlets(n_row,n_col,n_shearlet)
psu = np.array([cl.convolve_stack(ui,shearlets_adj) for ui in U]) #shealret adjoint of U, i.e Psi^{Star}(U)
mus = cl.comp_mu(psu)

/Users/fnammour/Documents/GitHub/score/cadmos_lib.py:104: ComplexWarning: Casting complex values to real discards the imaginary part
  coeff[s] = trafo.adjoint_transform(temp, do_norm=False)


### 1. Single Window SCORE Loss

In [22]:
def loss_SW_score(residual):
    data_fid = [norm(r)**2/2. for r in residual]
    print("data_fid=",data_fid)
    shape_constraint = gamma*(np.array([m*((r*w*u).sum())**2
                                        for r,m,u,w in zip(residual,muw,U,win)])/2.)
    print("shape_constraint=",shape_constraint)
    return data_fid+shape_constraint

print("loss=",loss_SW_score(residual))

data_fid= [1546.1110829319493, 1564.7831197659116, 1559.7051593505662]
shape_constraint= [ 998300.53418549 1027748.57909154 1367522.40986093]
loss= [ 999846.64526842 1029313.36221131 1369082.11502028]


### 2. Rewriting Single Window ShapeDecon Loss

In [13]:
def loss_SW_keras(y_pred,y_true):
    y_pred = tf.reshape(y_pred, [1,*y_pred.shape,1])
    y_true = tf.reshape(y_true, [1,*y_true.shape,1])
    residual = y_true-y_pred
    data_fid = tf.keras.backend.sum(tf.keras.backend.square(y_true-y_pred),axis=(1,2,3))/2.
    window = tf.reshape(win[0], [1,*win.shape[1:],1])
    U_tensor = tf.reshape(U, [U.shape[0],1,*U.shape[1:],1])
    mu = tf.reshape(muw, [*muw.shape])
    shape_constraint=0
    for i in range(6):
        shape_constraint+=gamma*mu[i]*\
        tf.keras.backend.square(
            (tf.keras.backend.sum(residual*window*U_tensor[i],axis=(1,2,3))))/2
    print("shape_constraint=",tf.keras.backend.get_value(shape_constraint))
    return data_fid+shape_constraint

print("loss=",[tf.keras.backend.get_value(loss_SW_keras(yp,yt)) for yp,yt in zip(y_pred,y_true)])

data_fid= [1546.11108293]
shape_constraint= [4983510.50694212]
data_fid= [1564.78311977]
shape_constraint= [5099637.54558196]
data_fid= [1559.70515935]
shape_constraint= [5098536.95031807]
loss= [array([4985056.61802505]), array([5101202.32870172]), array([5100096.65547742])]


### 3. Multi Window SCORE Loss

In [5]:
def loss_MW_score(residual):
    data_fid = np.linalg.norm(residual)**2/2.
    print("data_fid=",data_fid)
    shape_constraint = gamma*(np.array(\
            [[mu_ij*((residual*psu_ij).sum())**2\
              for mu_ij,psu_ij in zip(mu_j, psu_j)]\
            for mu_j,psu_j in zip(mus,psu)])/2.).sum()
    print("shape_constraint=",shape_constraint)
    loss = data_fid+shape_constraint
    return loss

print("loss=",loss_MW_score(residual))

data_fid= 1532.8932077213979
shape_constraint= 2646.1208671422955
loss= 4179.014074863693


### 4. Rewriting Multi Window ShapeDecon Loss

In [7]:
def loss_MW_keras(y_pred,y_true):
    y_pred = tf.reshape(y_pred, [1,*y_pred.shape,1])
    y_true = tf.reshape(y_true, [1,*y_true.shape,1])
    mu = tf.reshape(mus, [*mus.shape])
    psu_tensor = tf.reshape(psu, [*psu.shape[:2],1,*psu.shape[2:],1])
    residual=y_pred-y_true
    data_fid = tf.keras.backend.sum(tf.keras.backend.square(y_true-y_pred),axis=(1,2,3))/2.
    print("data_fid=",tf.keras.backend.get_value(data_fid))
    shape_constraint=0
    for i in range(6):
        for j in range(27):
            shape_constraint+=gamma*mu[i,j]*\
            tf.keras.backend.square(
                tf.keras.backend.sum(residual*psu_tensor[i,j],axis=(1,2,3)))/2.
    print("shape_constraint=",tf.keras.backend.get_value(shape_constraint))
    loss=data_fid+shape_constraint
    return loss

print("loss=",tf.keras.backend.get_value(loss_MW_keras(y_pred,y_true)))

data_fid= [1532.89320772]
shape_constraint= [2646.12086714]
loss= [4179.01407486]
